>## Import des librairies

In [ ]:
#Import des packages
import pandas as pd

#Tokenisation
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Sauvegarde
from joblib import dump,load

#API Google
!pip install google-cloud-translate==2.0.1
from google.cloud import translate_v2 as translate

key_path = 'rakuten-390308-9193868c526b.json'
client = translate.Client.from_service_account_json(key_path)

#import stem
from nltk.stem.snowball import FrenchStemmer
porter_stemmer = FrenchStemmer()

#import lemme
#à lancer selon l'environnement

!python -m spacy download fr_core_news_md

import spacy
nlp = spacy.load('fr_core_news_md')

!pip install Unidecode
from unidecode import unidecode

>## Fonctions de référence

In [ ]:
#fonction pour gestion de la ponctuation et des balises avant suppression
def remplacement(text):
  list_ponctuation = [",",".","?","-","!","/","\\","'",":","!"]
  #ajout d'un espace avant et après ponctuation pour éviter des erreurs de tokenisation
  for i in list_ponctuation:
    text = text.replace(i," "+i+" ")
  #ajout d'espace avant ou après selon chevron pour éviter concatenation de mots après retrait balise html
  text = text.replace("<"," <")
  text = text.replace(">","> ")
  #gestion des caractères mal encodés hors html (A RETRAVAILLER EN UTILISANT PYENCHANT)
  text = text.replace("à¢","â")
  text = text.replace("â¿¿","'")
  text = text.replace("Â°","°")
  text = text.replace("¿","oe")
  #suppression encodage html
  from bs4 import BeautifulSoup
  soup = BeautifulSoup(text, 'html.parser')
  text = soup.get_text()
  return text

def encodage_html_liste(liste):
  from bs4 import BeautifulSoup
  nouvelle_liste=[]
  for i in liste:
    nouvelle_sous_liste=[]
    for j in i:
      soup = BeautifulSoup(j, 'html.parser')
      nouvelle_sous_liste.append(soup.get_text() )
    nouvelle_liste.append(nouvelle_sous_liste)
  return nouvelle_liste


def tokenisation(texte):
  doc = nlp(texte)
  liste = [i.text for i in doc]
  return liste

#fonction de détection de langue avec API Google
def detect_language(text):
  result = client.detect_language(text)
  return result

#Fonction de traduction API Google
def translate_list(liste):
  resultat = []
  for i in liste:
    translated_text = client.translate(i, target_language='fr')
    resultat.append(translated_text['translatedText'])
  return resultat

#fonction de tokenisation d'une liste dans une liste
def tokenize(liste):
  resultat = []
  for i in liste:
    mot_tokenize =  word_tokenize(i.lower(), language = 'french')
    for k in mot_tokenize:
      resultat.append(k)
  return resultat

# fonction de retrait des mots vides (A REPRENDRE AVEC SPACY???)
def stop_word_spacy(liste):
  stop_words = spacy.lang.fr.stop_words.STOP_WORDS
  tokens = [i for i in liste if i not in stop_words]
  return tokens

def nettoyage(tokens):
  stop_words = set(stopwords.words('french'))
  result=[token for token in tokens if token not in stop_words]
  return result

#fonction de retrait des caractères non alpha-numeriques

def alphanum(tokens):
  result = [token for token in tokens if token.isalnum()]
  return result

#fonction de retrait des mots exclusivement numériques
def retrait_chiffres(tokens):
  result=[token for token in tokens if not (token.isnumeric ())]
  return result

def retrait_accents(liste):
  return [unidecode(i) for i in liste]

def retrait_mail(s):
  s = re.sub(r'http\S*','',s)
  return(s)

#fonction de lemmatisation avec SPACY
def lem_word(words):
  lemmes = [token.lemma_ for token in nlp(" ".join(words))]
  return lemmes

#fonction de stemming

def stemming(liste):
  result = [porter_stemmer.stem(i) for i in liste]
  return result


## Fonctions principales

In [ ]:
def pre_traduction(df):
  #création colonne description vide
  df['description_vide'] = df['description'].isna()
  # fusion de désignation et description
  df["texte"] = df.apply(lambda row: row["designation"] + " " + row["description"] if not (row["description_vide"] or (row["designation"] == row["description"])) else row["designation"], axis=1)
  #retrait des colonnes initiales description et designation
  to_drop=['description','designation']
  df = df.drop(to_drop,axis=1)
  #retrait des doublons dans texte
  df = df.drop_duplicates(subset='texte')
  #application fonction probleme encodage divers
  df["texte"] = df["texte"].apply(remplacement)
  #detection de langue
  df['langue_texte'] = df['texte'].apply(lambda x: detect_language(x)['language'])
  #conservation des langues "fr","en","es","it" et "de".
  df = df.loc[(df['langue_texte']=='fr')|(df['langue_texte']=='en')|(df['langue_texte']=='de')|(df['langue_texte']=='it')|(df['langue_texte']=='es')]
  #tokenisation
  df['texte'] = df['texte'].astype('str')
  df['texte_decoupe']=df['texte'].apply(lambda x: tokenisation(x.lower()))
  return df

In [ ]:
def traduction(df):
  df['texte_decoupe_traduit_fr']=df['texte_decoupe'].apply(translate_list)
  return df

In [ ]:
def post_traduction(df):
  #retrait des colonnes texte et texte découpé
  df = df.drop(columns=['texte','texte_decoupe'])
  #retrait des balises à une liste de liste de mots
  df["mots"] = df["mots"].apply(encodage_html_liste)
  #retokenisation
  df["mots"] = df["mots"].apply(tokenize)
  #retrait des mots vides
  df['mots'] = df['mots'].apply(stop_word_spacy)
  #retrait des accents
  df['mots'] = df['mots'].apply(retrait_accents)
  #suppression des caractères non alpha numériques
  df["mots"] = df["mots"].apply(alphanum)
  #détection et retrait adresse mail
  df['mots'] = df['mots'].apply(retrait_mail)
  #ajout de deux colonnes de stemming
  df["mots_stem"] =  df["mots"].apply(stemming)
  df["mots_stem_sans_chiffres"] =  df["mots_stem"].apply(retrait_chiffres)
  #ajout de deux colonnes de lemmatisation
  df['mots_lem'] = df['mots'].apply(lem_word)
  df["mots_lem_sans_chiffres"] =  df["mots_lem"].apply(retrait_chiffres)
  #ajout d'une colonne qui stem apres lem
  df["mots_lem_stem"] = df["mots_lem"].apply(stemming)
  df["mots_lem_stem_sans_chiffres"] = df["mots_lem_stem"].apply(retrait_chiffres)
  return df


>## Application des fonctions à X_train et X_test

Import X_train et X_test et fusion X_train avec y_train

In [ ]:
df_train = pd.read_csv("X_train_update.csv",index_col=0)
Y_train = pd.read_csv("Y_train_CVw08PX.csv",index_col=0)
#fusion X_train et y_train avant pretraduction
df_train = pd.concat([df_train,Y_train], axis=1)
X_test_update = pd.read_csv("X_test_update.csv",index_col=0)

Appliquer la Fonction pretraduction

In [ ]:
df_train_pretraduit = pre_traduction(df_train)
dump(df_train_pretraduit,'df_train_pretraduit.joblib')

df_test_pretraduit = pre_traduction(X_test_update)
dump(df_test_pretraduit,'df_test_pretraduit.joblib')


In [ ]:
df_test_pretraduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df_test_pretraduit.joblib")
df_train_pretraduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df_train_pretraduit.joblib")

Traduction Train

In [ ]:
#séparation du df selon langue française
df_train_non_fr = df_train_pretraduit.loc[df_train_pretraduit['langue_texte'] != 'fr']
df_train_fr = df_train_pretraduit.loc[df_train_pretraduit['langue_texte'] == 'fr']

#découpage du df non français pour procéder à une traduction par étapes
df1_train_non_fr = df_train_non_fr.iloc[0:2000,:]
df2_train_non_fr  = df_train_non_fr.iloc[2000:4000,:]
df3_train_non_fr  = df_train_non_fr.iloc[4000:6000,:]
df4_train_non_fr  = df_train_non_fr.iloc[6000:8000,:]
df5_train_non_fr  = df_train_non_fr.iloc[8000:10000,:]
df6_train_non_fr  = df_train_non_fr.iloc[10000:,:]


In [ ]:
#Appliquer la fonction traduction
df1_train_traduit = traduction(df1_train_non_fr)
dump(df1_train_traduit,'df1_train_traduit.joblib')
df2_train_traduit = traduction(df2_train_non_fr)
dump(df2_train_traduit,'df2_train_traduit.joblib')
df3_train_traduit = traduction(df3_train_non_fr)
dump(df3_train_traduit,'df3_train_traduit.joblib')
df4_train_traduit = traduction(df4_train_non_fr)
dump(df4_train_traduit,'df4_train_traduit.joblib')
df5_train_traduit = traduction(df5_train_non_fr)
dump(df5_train_traduit,'df5_train_traduit.joblib')
df6_train_traduit = traduction(df6_train_non_fr)
dump(df6_train_traduit,'df6_train_traduit.joblib')


In [ ]:
df1_train_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df1_train_traduit.joblib")
df2_train_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df2_train_traduit.joblib")
df3_train_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df3_train_traduit.joblib")
df4_train_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df4_train_traduit.joblib")
df5_train_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df5_train_traduit.joblib")
df6_train_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df6_train_traduit.joblib")

In [ ]:
#concaténation des parties
df_texte_non_fr_traduit = pd.concat([df1_train_traduit,df2_train_traduit,
                                     df3_train_traduit,df4_train_traduit,
                                     df5_train_traduit,df6_train_traduit],axis=0)
#fusion des textes non traduits et traduits
dict_1 = {'texte_decoupe_traduit_fr':'mots'}
df_texte_non_fr_traduit.rename(dict_1,axis=1,inplace=True)
dict_2 = {'texte_decoupe':'mots'}
df_train_fr.rename(dict_2,axis=1,inplace=True)
#fusion des deux df
df_train = pd.concat([df_train_fr, df_texte_non_fr_traduit],axis=0)
dump(df_train,'df_train.joblib')

Traduction TEST

In [ ]:
#séparation du df_test selon langue française
df_test_non_fr = df_test_pretraduit.loc[df_test_pretraduit['langue_texte'] != 'fr']
df_test_fr = df_test_pretraduit.loc[df_test_pretraduit['langue_texte'] == 'fr']

#découpage du df non français pour procéder à une traduction par étapes
df1_test_non_fr = df_test_non_fr.iloc[0:800,:]
df2_test_non_fr = df_test_non_fr.iloc[800:,:]

In [ ]:
df1_test_traduit = traduction(df1_test_non_fr)
dump(df1_test_traduit,'df1_test_traduit.joblib')
df2_test_traduit = traduction(df2_test_non_fr)
dump(df2_test_traduit,'df2_test_traduit.joblib')

In [ ]:
df1_test_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df1_test_traduit.joblib")
df2_test_traduit=load("/content/drive/MyDrive/Rakuten/_df_Traduit_Final/df2_test_traduit.joblib")

In [ ]:
#concaténation des parties
df_test_texte_non_fr_traduit = pd.concat([df1_test_traduit,df2_test_traduit],axis=0)
#fusion des textes non traduits et traduits
dict_1 = {'texte_decoupe_traduit_fr':'mots'}
df_test_texte_non_fr_traduit.rename(dict_1,axis=1,inplace=True)
dict_2 = {'texte_decoupe':'mots'}
df_test_fr.rename(dict_2,axis=1,inplace=True)
#fusion des deux df
df_test = pd.concat([df_test_texte_non_fr_traduit,df_test_fr],axis=0)
dump(df_test,'df_test.joblib')

Traitement post traduction

In [ ]:
df_train = load('df_train.joblib')
df_test = load('df_test.joblib')

In [ ]:
tokenize(test)

In [ ]:
df_train = post_traduction(df_train)
df_test = post_traduction(df_test)

>## Sauvegarde avant regex et ML

In [ ]:
dump(df_train,'df_train_post_trad.joblib')
dump(df_test,'df_test_post_trad.joblib')